In [ ]:
#loading necessary libraries
library(tidyverse)
library(recipes)


# Dataset Descriptions
## What is the dataset about?
This dataset was collected by the Pacific Laboratory for Artificial Intelligence (PLAI) led by Frank Wood at the University of British Columbia. The data collected revolves around how people play video games and was recorded through a MineCraft server ,set up by the research team, where players' actions are recorded as they navigate through a MineCraft world.  

#### players.csv
This original file contains 196 observations and 7 variables which are explained below  

1. **experience**: classifies a players experience into 5 categories: Pro, Veteran, Amateur, Regular and Beginner. The datatype is in character form.        
2. **susbcribe**: TRUE if player has subscribed to newsletter and FALSE otherwise. This column is of a logical type.  
3. **hashedEmail**: encrypted email address of player. This column is of the character type.  
4. **played_hours**: number of hours spent on the game by the player. The values in this column are a double type.  
5. **name**: name of player. Values in this column are of character type.  
6. **gender**: gender of player with unique values: Male , Female, Non-binary, Prefer not to say, Agender, Two-Spirited, Other. This column is of character type.   
7. **Age of player**: age of the player and values in this column are of double data type.



### Potential issues with players.csv 

1.The "experience" variable is in the character data type format; it should be changed to the factor data format.  

2.The "hashedEmail" variable should be deleted because it exposes players' identities.  

3.The "gender" variable is a character type data; it should be changed to the factor data type.  

4.The "name" variable should be deleted because it personally identifies players.
        

In [ ]:
download.file("https://raw.githubusercontent.com/tahsansamin/project_planning_stage_individual/refs/heads/main/dataset/sessions.csv", "sessions.csv")
download.file("https://raw.githubusercontent.com/tahsansamin/project_planning_stage_individual/refs/heads/main/dataset/players.csv", "players.csv")

In [ ]:
players <- read_csv("players.csv", show_col_types = FALSE)
players

#### sessions.csv
This original file consists of 1535 observations and the following variables:  

1. **hashedEmail** : hashed email address of players in character data type format
2. **start_time** : records date and time of when player started gaming session. Current data type in character format and it may need to be reformatted to date time format.
3. **end_time** : records date and time of when player stopped gaming session. Current data type in character format and it may need to be reformatted to date time format.
4.  **original_start_time** : unix timestamp (in seconds) of when player started session. The datatype in this column is of the double data type. 
5. **original_end_time** : unix timestampe (in seconds) of when player ended gaming session. The datatype in this column is of the double data type.
    
The difference between the original_end_time and the original_start_time can be used to easily calculate the duration of the playing session accurately than the start_time and end_time variables since the latter two could be in different timezones.

<b>Note: </b>The hashedEmail variable should be removed because it contains sensitive personal information (players' emails).

In [ ]:
sessions <- read_csv("sessions.csv", show_col_types = FALSE)
sessions

### Summary Statistics

#### Players.csv

| Variable | Minimum | Maximum | Mean |
| -------- | ------- | -------- | -----|
| Age | 9 | 58 | 21.14 |
| played_hours | 0 | 223.10 | 5.85 |

#### sessions.csv
Note: Some of the statistics for original_start_time and original_end_time have been reported to more than two decimal places because reporting them to two decimal places makes it seem that it is the same value as another summary statistic whereas the difference between the two is too minute to be displayed with two decimal places. However, do note that the minimum for both original_start_time and original_end_time are the same and hence have been reported to two decimal places.

| Variable | Minimum | Maximum | Mean |
| -------- | ------- | -------- | -----|
| original_start_time | 1.71e+12 | 1.72733e+12	 | 1.719201e+12	 |
| original_end_time | 1.71e+12 | 1.72734e+12 | 1.719196e+12 |


In [ ]:
summary_stats_players <- players |> 
                summarise(min_Age = min(Age, na.rm = TRUE),max_Age = max(Age, na.rm = TRUE),mean_age = mean(Age, na.rm = TRUE), min_played_hours = min(played_hours),max_played_hours = max(played_hours), 
                         , mean_played_hours = mean(played_hours, na.rm = TRUE))
summary_stats_players
summary_stats_sessions <- sessions |> 
                summarise(min_original_start_time = min(original_start_time, na.rm = TRUE), max_original_start_time = max(original_start_time, na.rm = TRUE), mean_original_start_time = mean(original_start_time, na.rm = TRUE),
                         min_original_end_time = min(original_end_time, na.rm = TRUE), max_original_end_time = max(original_end_time, na.rm = TRUE), mean_original_end_time = mean(original_end_time, na.rm = TRUE))

summary_stats_sessions

# Questions
The **broad question** that will be explored is:
        *What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ            between various player types?* More **specifically** *can player age and number of hours played be used to predict their newsletter subscription status?*

In order to answer this question the players.csv file will be used because it contains the player age, number of hours played and subscription status variables. First, the name and hashed email's of all the players will be removed to keep the data confidential. Also , the player experience variable will be changed to be a factor data format. Next, the rows that have NA or missing values for player age , number of hours played and newsletter subscription status will be removed because there are not many NA or missing values so removing them is very unlikely to cause loss of valuable data ( btw check this Samin!). After that, the hours played and age variables will be standardized so no one variable takes more weight in prediction that the other. Finally, the K nearest-neighbors algorithm will be applied to predict a player's newsletter susbcription status based on their age and the number of hours they have played.

# Explaratory Data Analysis and Visualizations

In [ ]:
na_vals <- colSums(is.na(players))
na_vals

Based on the above results, only two observations have NA values for Age. Because of such a small number of observations that have NA, these observations can be removed.

In [ ]:
tidy_players <- players |>
            
            drop_na()
tidy_players

#### Table Summarizing mean of quantitative variables
| Variable | Mean |
-----------|--------|
| played_hours | 5.90|
| Age | 21.14|  

Note: The mean of the Age variable after dropping the NA values changed from 5.85 to 5.90.

In [ ]:
mean_vals <- tidy_players |>
            select(played_hours, Age) |>
            summarize(across(played_hours:Age, mean))
mean_vals

### Visualization 1: Number of Hours played plotted against age
The y variable played_hours was plotted scaled with log because of it's abnormal range of values from 0 hours played all the way to a maximum of 223 hours. This would better allow any trends to be observed for our purposes  

Note: This was done according to the advice of a teaching assistant during office hours.

In [ ]:
figure_1 <- tidy_players |>
            ggplot(aes(x = Age, y= log1p(played_hours), color = subscribe)) +
            geom_point() +
            labs(x = "Age of Player", y = "Number of Hours Played", title = "Number of Hours played against Age of Player") +
            theme(text = element_text(size = 14)) 

figure_1

The visualization shows a trend that younger players have higher number of hours played. It also seems that the observations with the highest number of hours played are all subscribed. However there seems to be a data imbalance with fewer observations for higher ages and fewer observations where the susbcription status is false.  This is will have to be dealt with so as not to bias the KNN model.

### Visualization 2: age of players who are susbcribed versus age of players who are not susbcribed

In [ ]:

hist_1 <- filter(tidy_players, subscribe == TRUE) |>
            ggplot(aes(x = Age)) +
            geom_histogram(fill = "#5B86AE") +
            
            labs(x = "Age of players", title = "Histogram for Age of players who are subscribers")+
            theme(text = element_text(size = 20))

hist_2 <- filter(tidy_players, subscribe == FALSE) |>
            ggplot(aes(x = Age)) +
            geom_histogram(fill = "#5B86AE") +
            labs(x = "Age of players", title = "Histogram for Age of players who are not subscribers") +
            theme(text = element_text(size = 20))
            

hist_1
hist_2

This histograms above highlight that players who are susbcribed are mostly in their late teens to early twenties(histogram 1). Majority of players who are not susbcribed tend to be younger than the late teens period (histogram 2).

### Visualization 3: Mean number of hours played for each subscription group

In [ ]:
means_for_subscription_status <- tidy_players |>
            group_by(subscribe) |>
            summarise(mean_hours  = mean(played_hours)) |> 
            ggplot(aes(x = subscribe, y = mean_hours)) +
            geom_bar(stat = "identity", fill = "#5B86AE") +
            geom_text(aes(label = format(round(mean_hours, 2)), figures = 2), vjust = 0, size = 6)+
            labs(x = "Subscription status", y = "Mean number of hours played", title = "Mean number of hours played for each subscription group")+
            theme(text = element_text(size = 20))

means_for_subscription_status


The above graph indicates that players who are subscribed have a much higher mean playing time versus players who are not subscribed (7.88 hours versus 0.50 hours)

### Visualization 4: Mean age for each subscription status

In [ ]:
means_for_subscription_status0 <- tidy_players |>
            group_by(subscribe) |>
            summarise(mean_age  = mean(Age)) |> 
            ggplot(aes(x = subscribe, y = mean_age)) +
            geom_bar(stat = "identity", fill = "#5B86AE") +
            geom_text(aes(label = format(round(mean_age, 2)), figures = 2), vjust = 0, size = 6)+
            labs(x = "Subscription status", y = "Mean age", title = "Mean age for each subscription group")+
            theme(text = element_text(size = 20))

means_for_subscription_status0

The graph shows a small difference between mean age for those subscribed versus those who are not. This could potentially be due to fewer observations in the dataset for older players

In [ ]:
#delete viz 2 and 3

# Methods and Planning

For this project a the K nearest neighbor algorithm will be used for classification. This is appropriate because the two variables of interest(hours played and age of player) are numerical. Also predicting the subscription status of a player is a classification problem so KNN classification would be appropriate. Furthermore, KNN is baesd on similarity of nearby points, so general trends in player behaviour patterns can be better understood. Another crucial aspect is that the KNN algorithm requires few assumptions of what data must look like.

##### Assumptions of the model
The KNN algorithm has few assumptions. Noteably, it assumes that the closer two given data points are the more related and similar they are to each other

##### Limitations of KNN classification
First, KNN is computationally efficient for large datasets which means that it would take more time to train. Furthermore, KNN classification may not perform well if the classes are imbalanced such as if the dataset constitutes more subscribers than subscribers. In this case the KNN algorithm would in many cases predict the class of the data point as the class with the highest number of occurences which may not always be right. 

##### How to compare and select the model?
5 fold cross validation with a range of values of K will be used to select the model based on the model with the highest cross validation accuracy.

##### Data preprocessing
The dataset players.csv will be split into a training and testing set with training data taking 75% of the original data and testing taking the remaining 25%. Splitting will be performed after removing confidential information from data, removing NA values and changing data type of some columns to another more appropriate data type.  Cross validation will be performed with 5 folds and a range of K values to determine which K is best.